In [ ]:
#r "nuget: Microsoft.SemanticKernel"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010

// Initialize the Semantic kernel
var kernelBuilder = Kernel.CreateBuilder();
var kernel = kernelBuilder
    .AddOpenAIChatCompletion(                        // We use Semantic Kernel OpenAI API
        modelId: "phi3:mini",
        apiKey: null,
        endpoint: new Uri("http://localhost:11434")) // With Ollama OpenAI API endpoint
    .Build();

var phi3 = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
var systemMessage = 
"""
You are a sports moderator.
You are realy enthusiastic about sport and have a lot of knowledge about it.
You also tend to overreact to things.
But you also have a good sense of humor.
""";

var chat = new ChatHistory(systemMessage);

In [ ]:
Func<string, Task> Chat = async (string input) => {
    Console.WriteLine($"User: {input}");
    chat.AddUserMessage(input);

    var assistantReply = await phi3.GetChatMessageContentAsync(chat, new OpenAIPromptExecutionSettings());
    chat.AddAssistantMessage(assistantReply.Content);

    Console.WriteLine($"\nBot:");
    Console.WriteLine($"{assistantReply}\n");
};

In [ ]:
#pragma warning disable SKEXP0001

while (true)
{
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    await Chat(userMessage);
}